In [1]:
import fire
import json
import os
import numpy as np
import tensorflow as tf
import tqdm
from tensorflow.core.protobuf import rewriter_config_pb2
import glob
import pickle

import model
from encoder import get_encoder

tf.__version__

'2.0.0-beta1'

In [2]:
cd /tf/src/data/gpt-2/

/tf/src/data/gpt-2


In [ ]:
!pip3 install -r requirements.txt

# Sample from Model

In [7]:
def sample_sequence(*, hparams, length, start_token=None, batch_size=None, context=None, temperature=1, top_k=0):
    if start_token is None:
        assert context is not None, 'Specify exactly one of start_token and context!'
    else:
        assert context is None, 'Specify exactly one of start_token and context!'
        context = tf.fill([batch_size, 1], start_token)

    def step(hparams, tokens, past=None):
        lm_output = model.model(hparams=hparams, X=tokens, past=past, reuse=tf.compat.v1.AUTO_REUSE)

        logits = lm_output['logits'][:, :, :hparams.n_vocab]
        presents = lm_output['present']
        presents.set_shape(model.past_shape(hparams=hparams, batch_size=batch_size))
        return {
            'logits': logits,
            'presents': presents,
        }

    def body(past, prev, output):
        next_outputs = step(hparams, prev, past=past)
        logits = next_outputs['logits'][:, -1, :]  / tf.cast(temperature, dtype=tf.float32)
        samples = tf.random.categorical(logits=logits, num_samples=1, dtype=tf.int32)
        return [
            next_outputs['presents'] if past is None else tf.concat([past, next_outputs['presents']], axis=-2),
            samples,
            tf.concat([output, samples], axis=1)
        ]

    past, prev, output = body(None, context, context)

    def cond(*args):
        return True

    _, _, tokens = tf.while_loop(
        cond=cond, body=body,
        maximum_iterations= 10 * length - 1,
        loop_vars=[
            past,
            prev,
            output
        ],
        shape_invariants=[
            tf.TensorShape(model.past_shape(hparams=hparams, batch_size=batch_size)),
            tf.TensorShape([batch_size, None]),
            tf.TensorShape([batch_size, None]),
        ],
        back_prop=False,
    )

    return tokens

# Generate Unconditionally

In [8]:
def main(
    model_name='m1',
    seed=None,
    nsamples=10,
    batch_size=1,
    length=None,
    temperature=1,
    top_k=0,
    models_dir='checkpoint',
):
    """
    Run the sample_model
    :model_name=117M : String, which model to use
    :seed=None : Integer seed for random number generators, fix seed to
     reproduce results
    :nsamples=0 : Number of samples to return, if 0, continues to
     generate samples indefinately.
    :batch_size=1 : Number of batches (only affects speed/memory).
    :length=None : Number of tokens in generated text, if None (default), is
     determined by model hyperparameters
    :temperature=1 : Float value controlling randomness in boltzmann
     distribution. Lower temperature results in less random completions. As the
     temperature approaches zero, the model will become deterministic and
     repetitive. Higher temperature results in more random completions.
    :top_k=0 : Integer value controlling diversity. 1 means only 1 word is
     considered for each step (token), resulting in deterministic completions,
     while 40 means 40 words are considered at each step. 0 (default) is a
     special setting meaning no restrictions. 40 generally is a good value.
     :models_dir : path to parent folder containing model subfolders
     (i.e. contains the <model_name> folder)
    """
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    enc = get_encoder("117M", "models")
    hparams = model.default_hparams()

    if length is None:
        length = hparams.n_ctx
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.compat.v1.Session(graph=tf.Graph()) as sess:
        np.random.seed(seed)
        tf.compat.v1.set_random_seed(seed)

        output = sample_sequence(
            hparams=hparams, length=length,
            start_token=enc.encoder['<|endoftext|>'],
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )[:, 1:]

        saver = tf.compat.v1.train.Saver()
        ckpt = tf.compat.v1.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        generated = 0
        while nsamples == 0 or generated < nsamples:
            out = sess.run(output)
            for i in range(batch_size):
                samples = enc.decode(out[i]).split("<|endoftext|>")
                for i, sample in enumerate(samples):
                    generated += 1
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    with open("/tf/src/data/gpt-2/samples/unconditional/m1/sample-" + str(generated), 'w') as f:
                        f.write(sample)

if __name__ == '__main__':
    main()


======================================== SAMPLE 1 ========================================
======================================== SAMPLE 2 ========================================
======================================== SAMPLE 3 ========================================


KeyboardInterrupt: 